In [2]:
import solana
from solana.publickey import PublicKey
from solana.rpc.api import Client
from solana.keypair import Keypair
from solana import system_program
from solana import transaction
from solana.system_program import SYS_PROGRAM_ID, CreateAccountParams, create_account
from solana.transaction import *
from spl.token.constants import TOKEN_PROGRAM_ID, ASSOCIATED_TOKEN_PROGRAM_ID
from spl.token.instructions import initialize_mint, InitializeMintParams
from spl.token._layouts import MINT_LAYOUT
from spl.token.client import Token
from spl.token import instructions
import base64
from borsh_construct import *
from solana.rpc.commitment import Commitment
import time
from solana.rpc.types import TokenAccountOpts

client = Client("https://api.devnet.solana.com")

client.is_connected()

True

In [8]:
program_id = PublicKey("5hLihu5RjNSTPFFGP2nurV3xnJCeLj12WJXgphRDVMvE")
# program_id_account_meta = AccountMeta(program_id, False, False)
sys_program_meta = AccountMeta(SYS_PROGRAM_ID, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)

In [9]:
def getAccountInfo(client: Client, account_pk: PublicKey):
    """Fetch account information from RPC, parse out the data and deserialize"""
    comm = Commitment("confirmed")
    res = client.get_account_info(account_pk, comm, encoding='base64')
    data = res['result']
    if isinstance(data, dict):
        return accountSchema.parse(base64.urlsafe_b64decode(data['value']['data'][0]))
    else:
        raise AttributeError(f'Unknown RPC result {data}')

In [4]:
seed = bytes([125,215,212,54,213,98,45,123]*4)
payer_keypair = Keypair.from_seed(seed)
# client.request_airdrop(payer_keypair.public_key, 2*10**9)

In [11]:
all_mints = [PublicKey(account['pubkey']) for account in client.get_token_accounts_by_owner(payer_keypair.public_key, TokenAccountOpts(program_id = PublicKey("TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA")), "max")['result']['value']]

accountSchema = CStruct(
    "date_created" / U32,
    "unlockable_date" / U32,
    "numeration" /U32,
    "rarity" / U8,
    "level" / U8,
    "xp" / U32,
    "rented_state" / Bool,
    "use_authority" / U8[32],
    "rent_bump" / U32,
)

avatar_program_id = PublicKey("DvTvXKZR9wveQzL3pBM7QJ9YZpConSXBXRTyDsEADNqX")

for mint in all_mints:
    try:
        (avatar_data_pda, _) = PublicKey.find_program_address([b'avatar_data_pda', bytes(mint)], avatar_program_id)
        print(PublicKey(getAccountInfo(client,avatar_data_pda).use_authority))
    except Exception as e:
        pass
        # print(e)



In [5]:
client.get_token_accounts_by_owner(payer_keypair.public_key, TokenAccountOpts(program_id = PublicKey("TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA")), "max")

{'jsonrpc': '2.0',
 'result': {'context': {'slot': 115530750},
  'value': [{'account': {'data': ['oljr41NH55ZmPXLz6E4t+6cUDJ2MA6sKisgkQH9xa1gRRzxJWTp6EpxMNRP55v7gGqpdbjkmO08CWQwxOOknGwEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA',
      'base64'],
     'executable': False,
     'lamports': 2039280,
     'owner': 'TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA',
     'rentEpoch': 266},
    'pubkey': 'J4Aa7KXfAoMMVvDyQsVHRQRrNjtEyvFRZPr7C1ycT28t'},
   {'account': {'data': ['fgZX9kquDOLbjwGTxbsg4bnX0KQqaHXE8eyUsf6fZSwRRzxJWTp6EpxMNRP55v7gGqpdbjkmO08CWQwxOOknGwEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA',
      'base64'],
     'executable': False,
     'lamports': 2039280,
     'owner': 'TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA',
     'rentEpoch': 267},
    'pubkey': '4o3jqYsWt4cPN4Au4yVaR7tuzfx4ggAAfpDPbUdNjHV5